In [1]:
import pandas as pd
from rich import print

In [2]:
catfile = '../gfcat/catalog/gfcat_object_table.csv'
vcatfile = '../gfcat/catalog/gfcat_visit_table.csv'
# Note that GFCAT includes a visit-level table that provides the eclipse numbers
# But we're going to pretend it doesn't exist for now so that we can show how to
#  search for eclipses from source positions in order to build the gPhoton2 input.
catdata = pd.read_csv(catfile)
vcatdata = pd.read_csv(vcatfile)

# Get the position and eclipse of the 5 most dramatic eclipse-like observations from GFCAT
vcatdata["mag_diff"] = pd.to_numeric(vcatdata["mag_diff"], errors="coerce")
objs = vcatdata[vcatdata['morphology']=='E'].nlargest(3,'mag_diff')
objs[['eclipse','ra','dec','simbad_otype','mag_diff']]

,eclipse,ra,dec,simbad_otype,mag_diff
632,15733,189.763271,65.825880,**,4.151942
628,15449,162.750488,5.425298,EclBin,3.057187
368,9900,135.096777,43.803453,blue,3.041757


In [3]:
from gPhoton.search.core import galex_cone_search
? galex_cone_search

Signature:
 galex_cone_search(
    ra: float,
    dec: float,
    arcseconds=2250,
    legs=False,
    aspect_dir: None | str | pathlib.Path = None,
)
Docstring:
This search function provides information on _possible_ observational coverage
of the provided source position (RA and Dec in decimal degrees) by looking for time
ranges in which the spacecraft boresight position as defined in the refine aspect
table falls within 0.625 degrees (or 2250 arcsecs) of the source position.

The returned values (in a pandas DataFrame) include eclipse number, object type,
the min/max RA/Dec of the boresight during the eclipse, and the FUV detector
temperature (used in calibration).
File:      ~/github/gPhoton2/gPhoton/search/core.py
Type:      function

In [4]:
# Let's find all the visits to each of the GFCAT objects above
for ra,dec in objs[['ra','dec']].values:
    visits = galex_cone_search(ra,dec)
    # `observed_legs` is the algorithmically determined number of visit legs (distinct from planned) using only aspect data
    # `ok_exposure_time` is the algorithmically estimated raw exposure time for the visit (including all legs) using only aspect data
    print(visits[['eclipse','plan_type','observed_legs','ok_exposure_time']])

eclipse plan_type  observed_legs  ok_exposure_time
1892      3976       AIS           13.0            1315.0
6358      9252       GII            1.0            1102.0
11696    15732       GII            1.0            1259.0
11697    15733       GII            1.0            1547.0
16540    21081       GII            1.0            1353.0
21692    26866       GII            1.0            1461.0
21693    26867       GII            1.0            1473.0

eclipse plan_type  observed_legs  ok_exposure_time
11443    15449       GII            1.0            1573.0
11591    15605       AIS           12.0            1245.0
15966    20459       AIS           12.0            1283.0
25386    31263       MIS            1.0            1699.0
29898    36378       MIS            1.0            1632.0

eclipse plan_type  observed_legs  ok_exposure_time
6836      9900       MIS            1.0            1641.0
15194    19631       AIS           12.0            1324.0
15820    20312       AIS           12.0            1273.0

gPhoton2 accepts an input catalog formatted as a `csv` file with the required columns of:
| Column Name | Units | Definition |
|-------------|-------|------------|
| `eclipse`   | #     | GALEX orbital eclipse number (count) |
| `ra`        | decimal degrees | Source or target right ascension (J2000) |
| `dec`       | decimal degrees | Source or target declination (J2000) |

Other columns can be present, but they will simply be ignored by gPhoton2.

> Note: This is major a change in behavior from `gPhoton1`, which would accept a source position _only_, and then automatically process all available data for that position. `gPhoton2` always requires an eclipse number. If an eclipse number is given with no target catalog, it will identify and extract photometry for all sources in the eclipse. If a target catalog is given, then it will extract photometry only at the locations and times specified, with any automated source detection.

In [5]:
# Construct the input catalog
# Limit our search to only MIS-like observations (==1 leg and >=600 seconds raw exposure)
testcat_frames = []
for ra,dec in objs[['ra','dec']].values:
    visits = galex_cone_search(ra,dec)
    visits_filtered = visits[(visits['observed_legs']==1) & (visits['ok_exposure_time']>=600)]
    visits_filtered['ra'] = ra
    visits_filtered['dec'] = dec
    testcat_frames.append(visits_filtered[['eclipse','ra','dec','ok_exposure_time']])
    #print(visits_filtered[['eclipse','plan_type','observed_legs','ok_exposure_time']])
testcat = pd.concat(testcat_frames, ignore_index=True)
testcat.to_csv('test_catalog.csv')
testcat

,eclipse,ra,dec,ok_exposure_time
0,9252,189.763271,65.825880,1102.0
1,15732,189.763271,65.825880,1259.0
2,15733,189.763271,65.825880,1547.0
3,21081,189.763271,65.825880,1353.0
4,26866,189.763271,65.825880,1461.0
5,26867,189.763271,65.825880,1473.0
6,15449,162.750488,5.425298,1573.0
7,31263,162.750488,5.425298,1699.0
8,36378,162.750488,5.425298,1632.0
9,9900,135.096777,43.803453,1641.0


In [6]:
testcat.__class__

pandas.DataFrame

In [7]:
from gPhoton.pipeline import execute_pipeline
? execute_pipeline

Signature:
 execute_pipeline(
    eclipse: int,
    band: Literal['NUV', 'FUV'],
    depth: int | None = None,
    threads: int | None = None,
    local_root: str = 'test_data',
    remote_root: str | None = None,
    download: bool = True,
    recreate: bool = False,
    verbose: int = 2,
    source_catalog_file: str | None = None,
    write: Mapping = mappingproxy({'image': True, 'movie': True}),
    aperture_sizes: Sequence[float] = (12.8,),
    lil: bool = True,
    coregister_lightcurves: bool = False,
    stop_after: Literal['photonpipe', 'moviemaker'] | None = None,
    compression: Literal['none', 'gzip', 'rice'] = 'gzip',
    hdu_constructor_kwargs: Mapping = mappingproxy({}),
    min_exptime: float | None = None,
    photometry_only: bool = False,
    burst: bool = False,
    chunksz: int = 1000000,
    share_memory: bool | None = None,
    extended_photonlist: bool = False,
    extended_flagging: bool = False,
    aspect: Literal['aspect', 'aspect2'] = 'aspect',
    override

In [8]:
eclipse,ra,dec = testcat.iloc[0][['eclipse','ra','dec']].values

In [ ]:
# With a decent internet connection and computer, this will take ~90-120 seconds per eclipse
# If the raw6 files are already present, then it will take ~30 seconds per eclipse
# So all in, this cell will probably take ~10-30 minutes to run...
for eclipse in testcat['eclipse'].unique():
    for band in ['NUV','FUV']:
        execute_pipeline(
            eclipse, # GALEX orbital eclipse number
            band, # GALEX band of "[NF]UV"
            depth=120, # movie frame / light curve integration
            # integer; None to deactivate (default None)
            threads=3,
            # where to both write output data and look for input data
            #local_root=, # default with be test_data/
            # auxiliary remote location for input data
            # remote_root="/mnt/s3",
            recreate=True,
            source_catalog_file="test_catalog.csv",
            # list of floats; relevant only to lightcurve / photometry portion
            aperture_sizes=[12.8],
            # actually write image/movie products? otherwise hold in memory but
            # discard (possibly after performing photometry).
            write={"movie": True, "image": True},
            coregister_lightcurves=True, # Matches lightcurve timestamps across bands --- eases analysis
            # photonpipe, moviemaker, None (default None) --- useful for development and debugging
            stop_after=None,
            # only produce photometry and no other outputs
            photometry_only=False,
            # "none", "gzip", "rice" compression scheme
            compression="rice",
            # use array sparsification on movie frames?
            lil=True,
            # write movie frames as separate files
            burst=False, # debugging mode that produces a lot of very short integration images
            extended_photonlist=True, # write the equivalent of GALEX extended photon files (aka x-files)
            extended_flagging=False, # detect and flag extended sources, only in detection mode
            verbose=4,
            #single_leg=1, # process only the specified leg
            #photonlist_cols=
        )


starting timer
eclipse 9252 NUV  -- GII; 1 leg(s)
trange= ( 790320189.995 , 790321300.995 )                   
RA AVG: 189.22205702439882, DEC AVG: 66.35446302003702, ROLL AVG: 68.99840952503288
Loading raw6 file...                                        
18715896 events
Unpacking raw6 data...                                      
stim_coef0, stim_coef1 = -94245.82710038454, 0.00012571051622597417
                                                            
Runtime statistics:
 runtime		=	7.59 sec. = (0.13 min.)
  processed	   =   18471843 of 18715896 events.
rate		=	2465078.77 photons/sec.

7.6 elapsed seconds, restarting timer
Cross-band frame coregistration requested, but exposure time table at this depth for FUV was not found.
making images from test_data/e09252/e09252-nd-b00.parquet
indexing data and making WCS solution
making full-depth image
making count image frame
making artifact mask image frame
making dose map image frame
making row std image frame
making mean row image fram

In [11]:
!ls test_data/e09900/*

test_data/e09900/e09900-fd-b00.parquet
test_data/e09900/e09900-fd-f0120-b00-movie-photom-12_8.csv
test_data/e09900/e09900-fd-f0120-b00-movie-r.fits
test_data/e09900/e09900-fd-ffull-b00-image-r.fits
test_data/e09900/e09900-fd-raw6.fits.gz
test_data/e09900/e09900-nd-b00.parquet
test_data/e09900/e09900-nd-f0120-b00-movie-photom-12_8.csv
test_data/e09900/e09900-nd-f0120-b00-movie-r.fits
test_data/e09900/e09900-nd-ffull-b00-image-r.fits
test_data/e09900/e09900-nd-raw6.fits.gz
